In [7]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
from collections import Counter
import keras
import pickle
from keras.layers import Dense, Conv1D, MaxPool2D, Flatten, Dropout, Conv2D, MaxPooling1D, TimeDistributed, LeakyReLU, SeparableConv2D
from keras.layers import Input, Dense,Activation, LSTM, MaxPooling1D, Flatten, LeakyReLU, Dropout
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

from sklearn.preprocessing import Normalizer

df = pd.read_csv('//datc//opschaler//combined_gas_smart_weather_dfs//processed/all_dwellings_combined_hour.csv', delimiter='\t', parse_dates=['datetime'])
df = df.set_index(['datetime'])
df = df.dropna()


def getdata():
    "Getting saved features and labels"
    with open('features', 'rb') as fp:
        X=pickle.load(fp)

    with open('labels', 'rb') as fp:
        y=pickle.load(fp)
    return X,y

X,y=getdata()

X=sc.fit_transform(X)

X=Normalizer().fit_transform(X)

X=np.expand_dims(X,axis=2)

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.1)

In [9]:
input = Input(shape=(14,1))  #lstm network

lstm=LSTM(16, return_sequences=True)(input)
maxp=MaxPooling1D()(lstm)

lstm=LSTM(32, return_sequences=True)(maxp)
maxp=MaxPooling1D()(lstm)

flat=Flatten()(maxp)

d=Dense(64, activation='relu')(flat)

d=Dense(128, activation='relu')(d)

d=Dense(256, activation='relu')(d)

d=Dense(512, activation='relu')(d)

output = Dense(1, activation='linear')(d)

model = Model(inputs=input, outputs=output)

adam=Adam(lr=0.0001)

model.compile(optimizer=adam, loss='mse')

model.fit(X_train,y_train,batch_size=16, epochs=10,validation_data=(X_test,y_test))

model.save('lstm')

Train on 94894 samples, validate on 10544 samples
Epoch 1/10
94894/94894 [==============================] - 193s - loss: 0.9552 - val_loss: 0.9742
Epoch 2/10
94894/94894 [==============================] - 186s - loss: 0.9353 - val_loss: 0.9584
Epoch 3/10
94894/94894 [==============================] - 184s - loss: 0.9239 - val_loss: 0.9484
Epoch 4/10
94894/94894 [==============================] - 186s - loss: 0.9131 - val_loss: 0.9364
Epoch 5/10
94894/94894 [==============================] - 185s - loss: 0.9057 - val_loss: 0.9401
Epoch 6/10
94894/94894 [==============================] - 152s - loss: 0.9013 - val_loss: 0.9430
Epoch 7/10
94894/94894 [==============================] - 147s - loss: 0.8956 - val_loss: 0.9185
Epoch 8/10
94894/94894 [==============================] - 147s - loss: 0.8899 - val_loss: 0.9120
Epoch 9/10
94894/94894 [==============================] - 146s - loss: 0.8851 - val_loss: 0.9118
Epoch 10/10
94894/94894 [==============================] - 147s - loss: 0.880

In [12]:
from keras.models import load_model
model=load_model('lstm')

preds=model.predict(X_test)

import matplotlib.pyplot as plt

x=np.arange(len(preds))

plt.plot(x,preds)
plt.plot(x, y_test)

plt.xlabel('Testing Example')
plt.ylabel('Gas Power')

plt.legend(("Predicted", "Actual"), loc='best')

plt.title("Results of LSTM")
plt.show()

ResourceExhaustedError: OOM when allocating tensor with shape[256,512]
	 [[Node: dense_9_3/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=87654321, seed2=971105, _device="/job:localhost/replica:0/task:0/device:GPU:0"](dense_9_3/random_uniform/shape)]]

Caused by op 'dense_9_3/random_uniform/RandomUniform', defined at:
  File "/opt/jupyterhub/anaconda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/jupyterhub/anaconda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/jupyterhub/anaconda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/jupyterhub/anaconda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/opt/jupyt